<h2> Exploratory analysis of a good CNN arhitecture and choice of hyperparameters </h2>
<p> This file is just an example of one exploratory analysis of a CNN arhitecture </p>

In [ ]:
import numpy as np
import h5py
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Conv2D, BatchNormalization, Dense, Dropout, MaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
def load_dataset():

    f = h5py.File('Test_Set.h5', 'r')
    X_test = np.array(f['array'][:]) # your train set features
    f.close()
    
    f = h5py.File('Test_Label.h5', 'r')
    Y_test = np.array(f['array'][:]) # your train set features
    f.close()
    
    f = h5py.File('Train_Set.h5', 'r')
    X = np.array(f['array'][:]) # your train set features
    f.close()
    
    f = h5py.File('Train_Label.h5', 'r')
    Y = np.array(f['array'][:]) # your train set features
    f.close()
    
    return X, Y, X_test, Y_test

In [ ]:
# --------- Load the data ------------

X, Y, X_test, Y_test = load_dataset()

print('Total number of examples to train ', Y.shape[0])
print('Number of negative examples', len(np.where (Y == 1)[0]))

<p style = "color:orange; font-size:24px; text-align:center;"> Define the model </p>

In [ ]:
def Cancer_clasifier(input_shapes):
    
    X_input = Input(input_shapes)
    
    XX = Conv2D(8, kernel_size = (1,1), strides=(1, 1), padding = 'same', 
                kernel_initializer='glorot_uniform')(X_input)       
    XX = BatchNormalization(axis = -1)(XX)
    XX = Activation('relu')(XX)
    # XX = MaxPooling2D(2,2)(XX)
 

    for no_filters in [16, 16*2]:
        XX = Conv2D(no_filters, kernel_size = (4,4), strides=(1, 1), padding = 'same')(XX)
        XX = BatchNormalization(axis = -1)(XX)
        XX = Activation('relu')(XX)
        XX = MaxPooling2D(2,2)(XX)

    
    # --- Output Layer
    XX = Flatten()(XX)
    
    XX = Dense(100)(XX)
    XX = Activation('relu')(XX)
    XX = Dropout(0.5)(XX)
    
    XX = Dense(1, activation='sigmoid')(XX)
    
    model = Model(inputs = X_input, outputs = XX, name='Cancer_clasifier')
              
    return model

<p style = "color:orange; font-size:24px; text-align:center;"> Train the model </p>

In [ ]:
my_cancer_model = Cancer_clasifier(X.shape[1:])

optimizing_function = Adam(0.0001, beta_1=0.9, beta_2 = 0.999)

my_cancer_model.compile(optimizer = optimizing_function, loss='binary_crossentropy', metrics=['accuracy'])

history = my_cancer_model.fit(x = X, y = Y, validation_data=(X_test,Y_test), epochs = 10, batch_size = 100)

Evaluate model with confussion matrix

In [ ]:
Y_pred = my_cancer_model.predict(X_test)

cm = confusion_matrix(Y_test, Y_pred)
total = sum(sum(cm))
accuracy = (cm[0,0]+cm[1,1])/total

print('Confusion matrix is: \n', cm)

print('summary of the model')
my_cancer_model.summary()

<p style = "color:orange; font-size:24px; text-align:center;"> Plot the loss </p>

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'], 'b')
plt.plot(history.history['val_loss'], 'r')
plt.legend(['train', 'validation'], loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Batch = 100, LR_Adam = 0.0001, \n 8,16 and 32 filters with Last layer and Dropout 0.5')

# -- If you want to save the image uncomment this line
#plt.savefig('11_May_Batch100_LR0.0001_8_1x1_16&32_4x4_Dropout_PlusLastLayer.png')

In [ ]:
Optionaly save the model:

In [ ]:
'''
# serialize model to JSON
model_json = my_cancer_model.to_json()
with open("my_cancer_model_10May_50epochs.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
my_cancer_model.save_weights("my_cancer_model_10May_50epochs.h5")
'''